In [1]:
from rsatoolbox import vis
from rsatoolbox import rdm
import rsatoolbox
import rsatoolbox.data as rsd 
import rsatoolbox.rdm as rsr

import numpy as np
import os
import inspect
import scipy.io
from collections import defaultdict

from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image

from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import torchvision
import torchvision.models as models
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms

from util_function import ImageDataset, get_roi_mapping, seed_everything, image_visualization_transform

In [2]:
seed_everything()

True


In [3]:
data_dir = '/home/yuchen/human/tutorial'
parent_submission_dir = '/home/yuchen/psy221f_project'

batch_size = 32
input_img_dim = (64,64)

device = 'cuda'

subj = 1
roi = ["V1v", "V1d", "V2v", "V2d", "hV4",
       "EBA", "FBA-1", "FBA-2",
       "OFA", "FFA-1", "FFA-2",
       "OPA", "PPA", "RSC","OWFA", "VWFA-1", "VWFA-2", ]

freeze_weights = True  # if True, freeze the visual encoder's weights
subset_training_data = False  # if True, only select a small proportion of the data for training

In [4]:
device = torch.device(device)


class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):

    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)


args = argObj(data_dir, parent_submission_dir, subj)

train_img_dir  = os.path.join(args.data_dir, 'training_split', 'training_images')

train_img_list = os.listdir(train_img_dir)
train_img_list.sort()

num_train = int(np.round(len(train_img_list) / 100 * 90))
idxs = np.arange(len(train_img_list))
np.random.shuffle(idxs)
idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]

print('Training stimulus images: ' + format(len(idxs_train)))
print('\nValidation stimulus images: ' + format(len(idxs_val)))

train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))


Training stimulus images: 8857

Validation stimulus images: 984


In [5]:
data_dir = args.data_dir
# idxs = idxs_val 
idxs = idxs_train


train_list = np.array(train_img_list)[idxs]
train_list = train_list[:1000]
        
fmri_dir = os.path.join(data_dir, 'training_split', 'training_fmri')
        
lfmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rfmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))
   
fmri = np.concatenate((lfmri[:,np.where(get_roi_mapping(data_dir, 'left', roi[0]))[0]], 
                      rfmri[:,np.where(get_roi_mapping(data_dir, 'right', roi[0]))[0]]),axis=1)
print(roi[0], fmri.shape[1])    
            
for i in roi[1:]:
  temp = np.concatenate((lfmri[:,np.where(get_roi_mapping(data_dir, 'left', i))[0]], 
                        rfmri[:,np.where(get_roi_mapping(data_dir, 'right', i))[0]]),axis=1)
  print(i, temp.shape[1])

  fmri = np.concatenate((fmri, temp),axis=1)
            
fmri = fmri[idxs]

print('fmri.shape', fmri.shape)


V1v 1154
V1d 1819
V2v 1519
V2d 1417
hV4 1296
EBA 6237
FBA-1 780
FBA-2 856
OFA 737
FFA-1 882
FFA-2 629
OPA 4669
PPA 2202
RSC 1061
OWFA 907
VWFA-1 1778
VWFA-2 892
fmri.shape (8857, 28835)


In [6]:
mkdir /home/yuchen/brain_rdm_new/

mkdir: cannot create directory ‘/home/yuchen/brain_rdm_new/’: File exists


In [7]:
from torchvision.models import resnet50

activity_map ={
    'V1': (0,2973),
    'V2': (2973, 5909),
    'V4': (5909, 7205),
    'EBA': (7205,13442 ),
    'FBA': (13442, 15078),
    'OFA':(15078, 15815),
    'FFA':(15815, 17326),
    'OPA':(17326, 21995),
    'PPA':(21995, 24197),
    'RSC':(24197, 25258),
    'OWFA':(25258, 26165),
    'VWFA':(26165, 28835)
}

brain_layer_dir = '/home/yuchen/brain_rdm_new/'

for layer_idx, name in enumerate(activity_map.keys()):
    activity = fmri[:, activity_map[name][0]:activity_map[name][1]]
    num_neurons = activity.shape[1]
    # measurements = activity
    measurements = activity[:1000,: ]
    
    nCond = measurements.shape[0]
    nVox = measurements.shape[1]

    des = {'session': 1, 'subj': 1}
    obs_des = {'conds': np.array(['cond_%02d' % x for x in np.arange(nCond)])}
    chn_des = {'voxels': np.array(['voxel_' + str(x) for x in np.arange(nVox)])}
    data = rsd.Dataset(measurements=measurements,
                    descriptors=des,
                    obs_descriptors=obs_des,
                    channel_descriptors=chn_des)

    RDM_euc = rsr.calc_rdm(data, descriptor='conds')
    dist_matrix_brain = RDM_euc.get_matrices()
    
    np.save(f'{brain_layer_dir}{name}.npy', dist_matrix_brain)
